In [8]:
#####################################################
###         Select event generator:               ###
###             0 : QGSM                          ###
###             1 : EPOS                          ###
##################################################### 

EVENT_GENERATOR = 0

#####################################################
###         Select features type:                 ###
###             0 : Statistical features          ###
###             1 : Inverse Time-of-flight        ###
##################################################### 

FEATURES_TYPE = 1

#####################################################
###         Select configuration properties       ###
#####################################################

detector_in_radius = 0.025 #0.125          # Внутренний радиус детекторов
detector_out_radius = 0.25         # Внешний радиус детекторов
detector_distances = [4]  # Расстояния установки детекторов
time_accuracy = 0.05                # Точность с которой пишется время прилета в нс
n_angles = 16                       # На сколько сегментов детектор разбивается по углам
n_radii = 22                        # На сколько сегментов детектор разбивается по
maximum_hits = 150                   # Максимальное количество частиц, попавших в детекторы в одном событии. Для разбиения 8х4 следует ставить 80-100
n_lines = 3                         # Количество линий информации. Первая - время прилета, Вторая - номер ячейки детектора, Третья - номер детектора
randomCoordinate = False            # Если True - то координата будет выбрана из нормального распределения, False - столкновения в нуле
gauss_sigma = 0.30                  # Standard deviation для определения места столкновения (в метрах)
output_path = 'data/modeled/'
name_of_file = 'PD173_QGSM_Big_InvTime'         # Имя файла, в который будет записана информация. Записывается в папку output_path с расширением .dat
N_of_files = 2                    # Количество файлов исходных данных  (QGSM: 100; EPOS: 181)



# Set directory with input raw generator files
if EVENT_GENERATOR == 0:
    folder_path = 'data/source/'               # Путь в папку где лежат исходные данные QGSM
elif EVENT_GENERATOR == 1:
    # folder_path = 'data/sourceEPOS_full/'                 # Тест с форматом OSCAR1999a  BASIC mode по <200 событий, полные
    folder_path = 'data/sourceEPOS_2k/'                 # Тест с форматом OSCAR1999a  BASIC mode по 2000 событий, только статус 0
else:
    raise Exception("Generator format should be chosen")

#####################################################
###        End of configuration properties        ###
#####################################################

import numpy as np

speed_of_light = 299792456      # m / s
proton_mass = 0.938272          # mass in GeV/c^2
masses_dict = {'211': 0.139570, '2212': 0.938272, '3112': 1.197449, '321': 0.493677, '3222': 1.18937,
              '3312': 1.32171, '3334': 1.67245, '-211': 0.139570, '-2212': 0.938272, '-3112': 1.197449, 
              '-321': 0.493677, '-3222': 1.18937, '-3312': 1.32171, '-3334': 1.67245}

charged_particles_list = ['211', '2212','3112', '321', '3222', '3312', '3334',
                          '-211', '-2212','-3112', '-321', '-3222', '-3312', '-3334']

# Данные взяты из EPOS 1 user guide
charged_particles_list_osc99a = ['120', '130', '240', '340', '150', '121', '131',
                          '241', '341', '151',
                          '-120', '-130','-240', '-340', '-150', '-121', '-131',
                          '-241', '-341', '-151',
                          '1120', '1130', '2230', '2330', '1140', '1240', '2140',
                          '1340', '3140', '1440', '2440', '3440', '1111', '2221', 
                          '1131', '2231', '3331', '1141', '1241', '1341', '1441',
                          '2441', '3441', '4441',
                          '-1120', '-1130', '-2230', '-2330', '-1140', '-1240', '-2140',
                          '-1340', '-3140', '-1440', '-2440', '-3440', '-1111', '-2221', 
                          '-1131', '-2231', '-3331', '-1141', '-1241', '-1341', '-1441',
                          '-2441', '-3441', '-4441',]

class Collision_event:
    def __init__(self, b, coordinate, particles) -> None:
        self.b = b
        self.coordinate = coordinate
        self.particles_list = particles
        self.moments = [0.0, 0.0, 0.0]

    def calculateMoments(self, det_distance) -> None:
        n = len(self.particles_list)
        momentSums = [0.0, 0.0, 0.0]
        for particle in self.particles_list:
            radius = np.sqrt(particle.px ** 2 + particle.py ** 2)
            angle = np.arctan2(radius, det_distance)
            for i in range(len(momentSums)):
                momentSums[i] += angle ** (i + 1)
        for i in range(len(momentSums)):
            if n > 0:
                self.moments[i] = momentSums[i] / n
    
    def calculateProtonMoments(self, det_distance) -> None:
        n = 0
        momentSums = [0.0, 0.0, 0.0]
        for particle in self.particles_list:
            if (particle.index == '2212' or particle.index == '-2212'):
                n += 1
                radius = np.sqrt(particle.px ** 2 + particle.py ** 2)
                angle = np.arctan2(radius, det_distance)
                for i in range(len(momentSums)):
                    momentSums[i] += angle ** (i + 1)
        for i in range(len(momentSums)):
            if n > 0:
                self.moments[i] = momentSums[i] / n

    def getParticlesInPseudorapidityBin(self, left_border, right_border) -> list:
        result = []
        for particle in self.particles_list:
            if left_border <= particle.pseudorapidity < right_border:
                result.append(particle)
        return result
            
    def getMultiplicityDensity(self, left_border, rigth_border) -> float:
        if left_border >= rigth_border:
            raise Exception("Left border must be less than right border")
        
        d_pseudorapidity = rigth_border - left_border

        particles = self.getParticlesInPseudorapidityBin(left_border, rigth_border)
        d_N = len(particles)
        return d_N / d_pseudorapidity

class Particle_hit:
    def __init__(self, index, time, px_normed, py_normed, pnorm, det_num) -> None:
        self.index = index
        self.time = time
        self.px_normed = px_normed
        self.py_normed = py_normed
        self.pnorm = pnorm
        self.det_num = det_num

# Определение массы частицы по ее индексу
def get_particle_mass(p_index: str) -> float:
    if len(p_index) != 10:
        return masses_dict[p_index]
    else:
        m = (int(p_index) % 10000) // 10
        return float(m * proton_mass)
        
# Вычисление скорости частицы по ее импульсу и индексу   
def calc_velocity(p: float, pz: float, ptype: str) -> float:
    return abs(pz) / np.sqrt(get_particle_mass(ptype) **2 + p**2 / 1**2)

# Вычисление скорости частицы по ее импульсу и массе  
def calc_velocity_by_mass(p: float, pz: float, pmass: str) -> float:
    return abs(pz) / np.sqrt(pmass **2 + p**2 / 1**2)

# Вычисление времени пролета по скорости и расстоянию до детектора
def calc_time(v: float, distance: float) -> float:
    c = 2.99722458 * 10**8
    velocity = v * c
    time = distance / velocity
    return time

# Нормировка времени пролета (t_light / t) на расстоянии current_distance
def normalize_time(t: float, dist: float):
    t_light = dist  * (10**9) / speed_of_light
    return  t_light / t

# Return is particle charged or not by its id
def isCharged(mc_id) -> bool:
    if len(mc_id) == 10 or mc_id in charged_particles_list:
        return True
    else:
        return False

def isCharged_osc99a(mc_id) -> bool:
    # if len(mc_id) == 9 or mc_id in charged_particles_list_osc99a:
    #     return True
    # else:
    #     return False
    return mc_id in charged_particles_list_osc99a


In [9]:
n_rings = len(detector_distances)
events_class = []
# Задать номера файлов с которыми будет идти работа
start_file = 1
final_file = N_of_files + start_file



for i in range(start_file, final_file):        #  iterate over all given files of simulated events    
    if EVENT_GENERATOR == 0:        # QGSM Format 
        filename = folder_path + "data_" + str(i) + ".dat"
    elif EVENT_GENERATOR == 1:      # EPOS Format
        filename = folder_path + "AuAu_11_5_dataOSC1999a_full_" + str(i) + ".data"
    else:
        raise Exception("Generator format should be chosen")

    with open(filename,'r') as inpf: 
        if EVENT_GENERATOR == 0:    # QGSM Format
            l = inpf.readline()
            l = inpf.readline()
            l = inpf.readline()
            l = inpf.readline()
            l = inpf.readline()
            for l in inpf.readlines():
                s=l.strip().split()
                if len(s) == 5:         # Start of the new event
                    b = float(s[2])
                    particle_hits = []

                    if randomCoordinate:
                        coordinate = np.random.normal(loc=0, scale=gauss_sigma)                                                     # Нормальное распределение
                        if abs(coordinate) > detector_distances[0]:
                            coordinate = 0
                    else: 
                        coordinate = 0   

                    events_class.append(Collision_event(b, coordinate, particle_hits))
                elif len(s) == 11:      # Particle line
                    p_charge = int(s[0].strip())
                    ptype = s[4].strip()

                    px = float(s[5])
                    py = float(s[6])
                    pz = float(s[7])
                    # pzlab = float(s[9])
                    # p_mass = float(s[10])

                    p_along = abs(pz)                                    
                    p_across = np.sqrt(px**2+py**2)
                    p_full = np.sqrt(p_across **2 + p_along **2)
                    
                    # Обработка всех частиц с ненулевым зарядом
                    if p_charge != 0:
                        for ind in range(n_rings):    # for every detector check if particle hit it
                            if pz > 0:
                                direction = 1
                                current_distance = detector_distances[ind] - coordinate
                            elif pz < 0:
                                direction = 0
                                current_distance = detector_distances[ind] + coordinate

                            is_under_upper_bound = (p_across / p_along) < (detector_out_radius / current_distance)
                            is_above_lower_bound = (p_across / p_along) > (detector_in_radius / current_distance)
                            is_hit_detector =  is_under_upper_bound and is_above_lower_bound

                            if is_hit_detector:                                     
                                det_number = 1 + ind * 2 + direction
                                pxn = px / p_full
                                pyn = py / p_full
                                p_velocity = calc_velocity(p_full, pz, ptype)
                                p_time = calc_time(p_velocity, current_distance)

                                particle_hits.append(Particle_hit(ptype, p_time, pxn, pyn, p_full, det_number))

        elif EVENT_GENERATOR == 1:      # EPOS (oscar1999a) Format
            l = inpf.readline()
            l = inpf.readline()
            l = inpf.readline()
            l = inpf.readline()
            for l in inpf.readlines():
                s=l.strip().split()
                if len(s) == 5:         # Start of the new event
                    b = float(s[3])
                    particle_hits = []
  
                    if randomCoordinate:
                        coordinate = np.random.normal(loc=0, scale=gauss_sigma)                                                     # Нормальное распределение
                        if abs(coordinate) > detector_distances[0]:
                            coordinate = 0
                    else: 
                        coordinate = 0   

                    events_class.append(Collision_event(b, coordinate, particle_hits))                                       #
                elif len(s) == 12:      # Particle line
                    ptype = s[1].strip()
                    # p_status_code = int(s[2])

                    # Обработка всех частиц с ненулевым зарядом 
                    if isCharged_osc99a(ptype):
                        px = float(s[3])
                        py = float(s[4])
                        pz = float(s[5])
                        # p_e = float(s[6])
                        p_mass = float(s[7])
                        # p_freeze_out_time = float(s[11])
                        
                        p_along = abs(pz)
                        p_across = np.sqrt(px**2+py**2)
                        p_full = np.sqrt(p_across **2 + p_along **2)

                        for ind in range(n_rings):    # for every detector check if particle hit it
                            if pz > 0:
                                direction = 1
                                current_distance = detector_distances[ind] - coordinate
                            elif pz < 0:
                                direction = 0
                                current_distance = detector_distances[ind] + coordinate


                            is_under_upper_bound = (p_across / p_along) < (detector_out_radius / current_distance)
                            is_above_lower_bound = (p_across / p_along) > (detector_in_radius / current_distance)
                            is_hit_detector =  is_under_upper_bound and is_above_lower_bound

                            if is_hit_detector:                                                                              
                                det_number = 1 + ind * 2 + direction 
                                pxn = px / p_full
                                pyn=py / p_full

                                p_velocity = calc_velocity_by_mass(p_full, pz, p_mass)
                                p_time = calc_time(p_velocity, current_distance)

                                particle_hits.append(Particle_hit(ptype, p_time, pxn, pyn, p_full, det_number))          


    print("File #{} has been read".format(i))
            


events_class = sorted(events_class, key=lambda x: (x.b))
print(f'{len(events_class)} events have been read')


File #1 has been read
File #2 has been read
4000 events have been read


In [10]:
n_detectors = 2*n_rings
detector_width = detector_out_radius - detector_in_radius
number_of_hits = np.zeros(3)
# Среднее время прилета пионов для детекторов на 1 метре, используется для нелинейного преобразования времени
avg_pion_time_1m = 3.345
max_hits = np.zeros(6, dtype=int)

# Разбиение детектора на n_radii одинаковых колец
one_ring_width = detector_width / n_radii
rings_width = []
# print(f'R = {detector_out_radius}; Nrad = {n_radii}; One Ring = {one_ring_width:.3f}')
for i in range(n_radii):
    rings_width.append(one_ring_width)

# Вычисление координат границ детекторов используя ширины колец
detector_borders = []
for i in range(len(rings_width)):
    rad = detector_in_radius
    for j in range(i, 0, -1):
        rad += rings_width[j - 1]
    detector_borders.append(rad)
detector_borders.append(detector_out_radius)
print(rings_width)
print("Границы радиального разбиения на {}".format(detector_borders))


n_radii = len(detector_borders) - 1

mean_radii = []
for i in range(n_radii):
    mean_radii.append( (detector_borders[i] + detector_borders[i + 1])/2 )
print("Средние значения радиусов: {}".format(mean_radii))


event_data=np.zeros((n_detectors, n_angles, n_radii), dtype=np.float32)
n = 0 

with open(f'{output_path}{name_of_file}.dat', 'w') as outf:
    if FEATURES_TYPE == 0:          # Statistical features
        outf.write('b, coordinate, time of first, first moment of angle, n particles \n')
    elif FEATURES_TYPE == 1:        # Time-of-flight features
        outf.write('Maximum hits: ' + str(maximum_hits) +  ' Lines: ' + str(n_lines) + ' \n')
    
    for e in events_class:    # iterate over events
        event_data_2 = []
        for i in range(n_lines): # листы: время, координата попадания, номер кольца
            event_data_2.append([])
        n += 1
        all_hits = sorted(e.particles_list, key=lambda x: (x.time))    # all hits for one event
        coordinate = e.coordinate
        b = e.b
        det_num = 0

        new_particle_angles = []
        new_n_particles = 0

        for hit in all_hits:    # Iterate over all particle hit
            # # Simulate detector efficiency
            # if np.random.random() > efficiency:
            #     continue
            p_type = hit.index
            det_value = hit.det_num
            det_num = (det_value - 1) // 2
            dir_num = (det_value - 1) % 2

            # Применяем нелинейное преобразование времени(записанного с заданной точностью), используя информацию о времени пролета пионов
            # Старый способ
            particle_time = time_accuracy * ((hit.time * 10**9)// time_accuracy)  
            particle_value = 1 / (particle_time - avg_pion_time_1m * detector_distances[det_num])                   # Changed                                             
            # particle_value = particle_time

            angle = np.arctan2(hit.px_normed, hit.py_normed)
            radius = np.sqrt(hit.px_normed ** 2 + hit.py_normed ** 2)
            angle_bin = np.floor(n_angles*(angle/2/np.pi+0.5))
            angle_bin = angle_bin.astype(int)
            if angle_bin >= n_angles:
                angle_bin -= 1

            # Вычисление радиальной координаты прилета частицы
            current_distance = detector_distances[det_num] + coordinate * (1 if dir_num == 0 else -1)
            rad = radius * current_distance
        
            isHit = False
            r_bin = 0
            # Определим бин по радиусу
            for i in range(len(detector_borders) - 1):
                if detector_borders[i] < rad <= detector_borders[i+1]:
                    r_bin = i
                    isHit = True
                    break

            # Если в эту ячейку еще не прилетала частица
            if event_data[det_num * 2 + dir_num, angle_bin, r_bin] == 0 and isHit:
                event_data[det_num * 2 + dir_num, angle_bin, r_bin] = particle_value

                event_data_2[0].append(particle_value)
                event_data_2[1].append(r_bin * n_angles + angle_bin + 1)
                event_data_2[2].append(det_value)

                                        
                new_n_particles += 1
                new_beam_angle = np.arctan2(radius, current_distance)
                new_particle_angles.append(new_beam_angle)

                number_of_hits[det_num] += 1

        # Ниже идет запись данных в файл
        # Проверка условия для отсеивания пустых событий
        if FEATURES_TYPE == 1:      # Time-of-flight features
            if event_data_2[0]:                                    # Отсеивание событий где ни одна частица не регистрируется
                evdata2 = np.array(event_data_2)

                event_data_new = np.array(np.c_[evdata2, np.zeros((n_lines, maximum_hits - evdata2.shape[1]))])                                           #
                output = np.reshape(event_data_new, (n_lines, 1, -1)).astype(np.float32)
                
                time_of_first = 0.0

                outf.write(str(b) + ' ' + str(coordinate) + ' ' + str(time_of_first) + ' ')
                for j in range(output.shape[0]):    # write into file
                    for i in range(output.shape[2]):
                        outf.write(str(output[j][0][i]))
                        outf.write(' ')
                    outf.write('   ')
                outf.write('\n')
                
                for j in range(output.shape[0]):    
                    max_hits[j] = max(np.count_nonzero(output[j]), max_hits[j])
                if n% int(len(events_class) / 10) == 0:
                    print("Записано {:3.0f}% событий".format(n * 100 / len(events_class)))
        elif FEATURES_TYPE == 0:    # Statistical features        
            allMoment = np.mean(new_particle_angles)   
            if str(allMoment) == 'nan':
                allMoment = 0
            if event_data_2[0]:                                    # Отсеивание событий где ни одна частица не регистрируется
                time_of_first = 0.0
                outf.write(str(b) + ' ' + str(coordinate) + ' ' + str(time_of_first) + '   ')

                if new_n_particles > 0:
                    outf.write(str(allMoment) + ' ' + str(new_n_particles))
                else:
                    outf.write(str(0) + ' ' + str(new_n_particles))

                outf.write('\n')

                if n% int(len(events_class) / 10) == 0:
                    print("Записано {:3.0f}% событий".format(n * 100 / len(events_class)))

        event_data*=0.0
        event_data = abs(event_data)
        # print("Максимальное количество попаданий по детекторам {}".format(max_hits))

[0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727, 0.010227272727272727]
Границы радиального разбиения на [0.025, 0.03522727272727273, 0.045454545454545456, 0.05568181818181818, 0.0659090909090909, 0.07613636363636363, 0.08636363636363635, 0.09659090909090907, 0.1068181818181818, 0.11704545454545452, 0.12727272727272726, 0.13749999999999998, 0.1477272727272727, 0.15795454545454543, 0.16818181818181815, 0.17840909090909088, 0.1886363636363636, 0.19886363636363633, 0.20909090909090905, 0.21931818181818177, 0.2295454545454545, 0.23977272727272722, 0.25]
Средние значения радиусов: [0.0

c:\Users\kiril\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\kiril\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
